Extract the subset of the data we want to look at.

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
sns.set()

# Read in the data

In [ ]:
raw_df = pd.read_csv("../data/rwjf/division_2022.csv")

In [ ]:
raw_df.head()

In [ ]:
raw_df.describe()

# Reformat the data

Let us change the structure of the data to make it easier to handle

In [ ]:
place_cols = ['FIPS', 'State', 'Division']
le_cols = [c for c in raw_df.columns if c.startswith('Life Expectancy (') and c.endswith(')')]
income_cols = [c for c in raw_df.columns if c.startswith('Household Income (') and c.endswith(')')]
pop_cols = ['Population', 
            '# Black', '% Black', 
            '# American Indian & Alaska Native', '% American Indian & Alaska Native', 
            '# Asian', '% Asian',
            '# Native Hawaiian/Other Pacific Islander',
            '% Native Hawaiian/Other Pacific Islander', 
            '# Hispanic', '% Hispanic',
            '# Non-Hispanic White', '% Non-Hispanic White',
            '% Female']

In [ ]:
def raw_to_place_df(raw_df):
    place_df = raw_df[place_cols]
    return place_df


raw_to_place_df(raw_df).head()

In [ ]:
def raw_to_le_df(raw_df):
    place_df = raw_to_place_df(raw_df)
    le_df = raw_df[le_cols]
    le_df.columns = ['AIAN', 'Asian', 'Black', 'Hispanic', 'White']
    le_df = place_df.join(le_df)
    le_df = le_df.set_index(place_cols)
    return le_df

raw_to_le_df(raw_df).head()

In [ ]:
def raw_to_pop_df(raw_df):
    place_df = raw_to_place_df(raw_df)
    pop_df = raw_df[pop_cols]
    pop_df = pop_df.drop(columns=["# Native Hawaiian/Other Pacific Islander", "% Native Hawaiian/Other Pacific Islander", "% Female"])
    pop_df.columns = ['Population', 
                      '# Black', '% Black', 
                      '# AIAN', '% AIAN',
                      '# Asian', '% Asian',
                      '# Hispanic', '% Hispanic',
                      '# White', '% White']
    pop_df = place_df.join(pop_df)
    pop_df = pop_df.set_index(place_cols)
    return pop_df


raw_to_pop_df(raw_df).head()

In [ ]:
def raw_to_income_df(raw_df):
    place_df = raw_to_place_df(raw_df)
    income_df = raw_df[income_cols]
    income_df.columns = ['AIAN', 'Asian', 'Black', 'Hispanic', 'White']
    income_df = place_df.join(income_df)
    income_df = income_df.set_index(place_cols)
    return income_df


raw_to_income_df(raw_df).head()

In [ ]:
def to_group_df(raw_df, group):
    pop_df = raw_to_pop_df(raw_df)
    le_df = raw_to_le_df(raw_df)
    income_df = raw_to_income_df(raw_df)
    df = pop_df[[f"# {group}", f"% {group}"]].join(le_df[group])
    df = df.join(income_df[group], rsuffix='_income')
    df.columns = ['population', 'percentage', 'life expectancy', 'income']
    df['group'] = group
    return df

In [ ]:
aian_df = to_group_df(raw_df, 'AIAN')
aian_df.head()

In [ ]:
groups = ['AIAN', 'Asian', 'Black', 'Hispanic', 'White']
df = pd.concat([to_group_df(raw_df, g) for g in groups])
df = df.sort_index()
df.head()

Let us save this division-level data

In [ ]:
df.to_csv("../data/viz/division_2022.csv", index=False)

And let us do the same for 2020

In [ ]:
raw_2020_df = pd.read_csv("../data/rwjf/division_2020.csv")
groups = ['AIAN', 'Asian', 'Black', 'Hispanic', 'White']
df_2020 = pd.concat([to_group_df(raw_2020_df, g) for g in groups])
df_2020 = df_2020.sort_index()
df_2020.head()

In [ ]:
df_2020.to_csv("../data/viz/division_2020.csv", index=False)

# Summarize data by state

Looking at county-level data is a bit overwhelming here, so let us summarize to the state level.

In [ ]:
def df_to_state_df(df):
    results = []
    for name, gdf in df.reset_index().groupby(["State", "group"]):
        tdf = gdf.dropna()
        pop_total = tdf['population'].sum()
        pop_frac =  tdf['population'] / pop_total
        result = {'State': name[0], 'group': name[1], 
                  'population': pop_total,
                  'life expectancy': (tdf['life expectancy'] * pop_frac).sum(),
                  'income': (tdf['income'] * pop_frac).sum()}
        results.append(result)
    state_df = pd.DataFrame(results)
    return state_df

In [ ]:
state_df = df_to_state_df(df)

In [ ]:
state_df.head()

There is something strange happening with the model for the AIAN. 

In [ ]:
tdf = state_df[state_df['life expectancy'] > 0]
sns.boxplot(x='group', y='life expectancy', data=tdf)

In [ ]:
fig, ax = plt.subplots()
tdf = state_df[state_df['life expectancy'] > 0]
col_order = tdf.groupby('group').mean().sort_values('life expectancy')
for i, group in enumerate(col_order.index):
    ttdf = tdf[tdf['group'] == group]
    ax.scatter(ttdf['group'], ttdf['life expectancy'], alpha=0.7)

So let is exclude that group for our the moment.

In [ ]:
fig, ax = plt.subplots()
tdf = state_df[state_df['group'] != 'AIAN']
tdf = tdf[tdf['life expectancy'] > 0]
col_order = tdf.groupby('group').mean().sort_values('life expectancy')
for i, group in enumerate(col_order.index):
    ttdf = tdf[tdf['group'] == group]
    ax.scatter(ttdf['group'], ttdf['life expectancy'], alpha=0.7)

This is the data we want to visualize, so let us export it.

In [ ]:
tdf.to_csv("../data/viz/state_2022.csv", index=False)

Same for 2020

In [ ]:
state_2020_df = df_to_state_df(df_2020)
state_2020_df.head()

In [ ]:
tdf = state_2020_df[state_2020_df['life expectancy'] > 0]
sns.boxplot(x='group', y='life expectancy', data=tdf)

It looks like the AIAN model improve in 2022...

In [ ]:
fig, ax = plt.subplots()
tdf = state_2020_df[state_2020_df['life expectancy'] > 0]
col_order = tdf.groupby('group').mean().sort_values('life expectancy')
for i, group in enumerate(col_order.index):
    ttdf = tdf[tdf['group'] == group]
    ax.scatter(ttdf['group'], ttdf['life expectancy'], alpha=0.7)

Export this as well

In [ ]:
tdf.to_csv("../data/viz/state_2020.csv", index=False)

# Income model

What is the relationship between income and life expectancy

In [ ]:
class LinReg:
    def __init__(self, df, xcol, ycol):
        self.df = df
        self.xcol = xcol
        self.ycol = ycol
        self.lm = None
        self.pred_range = None
        self.preds_input = None
        self.predictions = None

    def fit(self):
        df = self.df
        xcol= self.xcol
        lm = smf.ols(formula=f"{self.ycol} ~ {xcol}", data=df).fit()
        pred_range = (df[xcol].min(), df[xcol].max())
        preds_input = pd.DataFrame({xcol: pred_range})
        predictions = lm.predict(preds_input)
        self.lm = lm
        self.pred_range = pred_range
        self.preds_input = preds_input
        self.predictions = predictions

In [ ]:
reg_dfs = []
tdf = state_df[state_df['life expectancy'] > 0]
tdf = tdf.rename(columns={'life expectancy': 'le'})
tdf['income'] = tdf['income'] / 1000
col_order = tdf.groupby('group').mean().sort_values('le')
for i, group in enumerate(col_order.index):
    ttdf = tdf[tdf['group'] == group]
    model = LinReg(ttdf, 'income', 'le')
    model.fit()
    reg_df = pd.DataFrame({"x": model.pred_range, "y": model.predictions})
    reg_df['r2'] = model.lm.rsquared
    reg_df['slope'] = model.lm.params['income']
    reg_df['group'] = group
    reg_dfs.append(reg_df)
reg_df = pd.concat(reg_dfs)
reg_df

In [ ]:
tdf = state_df[state_df['life expectancy'] > 0]
col_order = tdf.groupby('group').mean().sort_values('life expectancy')
fig, axs = plt.subplots(1, len(col_order), sharex=True, sharey=True, figsize=(12, 3))
for i, group in enumerate(col_order.index):
    ttdf = tdf[tdf['group'] == group]
    ax = axs[i]
    ax.scatter(ttdf['income'] / 1000, ttdf['life expectancy'], alpha=0.5)
    ax.set_title(group)
    if i == 0:
        ax.set_ylabel('life expectancy')
        ax.set_xlabel('income (thousands)')
    ttdf = reg_df[reg_df['group'] == group]
    ax.plot(ttdf['x'], ttdf['y'])

Save this regression model

In [ ]:
reg_df.to_csv("../data/viz/regression_2022.csv", index=False)